In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
num_bootstraps = 10
train_ratio = 0.7
valid_ratio = 0.2
test_ratio = 0.1

In [ ]:
sts = pd.read_csv('/home/sn69/recipes_output/xref/sts-hd5/list_exactly_N_in_all_time_windows.csv', low_memory=False)
sts = sts.sort_values(['medrecn', 'surgdt'])
sts = sts.drop_duplicates(subset='medrecn', keep='last')
sts['surgdt'] = sts['surgdt'].apply(lambda dt: dt[:10]) #2000-01-01

In [ ]:
death_labels = sts[sts['mtopd'] == 1][['medrecn', 'surgdt', 'mtopd']]
no_death_labels = sts[sts['mtopd'] == 0][['medrecn', 'surgdt', 'mtopd']]

In [ ]:
bootstraps = []
for i in range(num_bootstraps):
    death_train, death_valid, death_test = np.split(death_labels.sample(frac=1).reset_index(drop=True), [int(train_ratio*len(death_labels)), int((train_ratio+valid_ratio)*len(death_labels))])
    no_death_train, no_death_valid, no_death_test = np.split(no_death_labels.sample(frac=1).reset_index(drop=True), [int(train_ratio*len(no_death_labels)), int((train_ratio+valid_ratio)*len(no_death_labels))])
    train = death_train.append(no_death_train).sample(frac=1).reset_index(drop=True)
    valid = death_valid.append(no_death_valid).sample(frac=1).reset_index(drop=True)
    test = death_test.append(no_death_test).sample(frac=1).reset_index(drop=True)
    bootstraps.append((train, valid, test))

In [ ]:
train_death = np.count_nonzero(train['mtopd'])
valid_death = np.count_nonzero(valid['mtopd'])
test_death = np.count_nonzero(test['mtopd'])
print(f'{train_death} / {len(train)} = {train_death/len(train):0.4f}')
print(f'{valid_death} / {len(valid)} = {valid_death/len(valid):0.4f}')
print(f'{test_death} / {len(test)} = {test_death/len(test):0.4f}')

In [ ]:
base_path = '/home/sn69/sts-ecg/bootstrap/'
for i, bootstrap in enumerate(bootstraps):
    i = str(i)
    os.mkdir(os.path.join(base_path, i))
    bootstrap[0].to_csv(os.path.join(base_path, i, 'train.csv'), index=False)
    bootstrap[1].to_csv(os.path.join(base_path, i, 'valid.csv'), index=False)
    bootstrap[2].to_csv(os.path.join(base_path, i, 'test.csv'), index=False)

In [ ]:
len(sts)